# Geoapify Melbourne Locations
## Police Stations

In [1]:
# Dependencies
import requests
import json
import pandas as pd

# Import the API key
from config import geoapify_key

In [2]:
# Set the parameters for the type of place - police stations
categories = "service.police"

# Set the parameters for the type of search of Melbourne (incl all Municipalities)
filters = "place:51397cf9ca8e286240593190f739f8ec42c0f00101f9016cca400000000000c002089203094d656c626f75726e65"
limit = 500

# Set up a parameters dictionary
params = {
    "categories": categories,
    "limit": limit,
    "filter": filters,
    "apiKey": geoapify_key
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Run a request using our params dictionary
response = requests.get(base_url, params=params)

In [3]:
# print the response url, avoid doing for public github repos in order to avoid exposing key
print(response.url)

https://api.geoapify.com/v2/places?categories=service.police&limit=500&filter=place%3A51397cf9ca8e286240593190f739f8ec42c0f00101f9016cca400000000000c002089203094d656c626f75726e65&apiKey=79372d4c67c34669935344d46743808f


In [4]:
# Convert response to JSON
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    145.22694819483766,
                    -37.81310875615223
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Ringwood Police Complex",
                "address_line2": "31 Ringwood Street, Ringwood VIC 3134, Australia",
                "categories": [
                    "building",
                    "building.facility",
                    "service",
                    "service.police",
                    "wheelchair",
                    "wheelchair.yes"
                ],
                "city": "Melbourne",
                "country": "Australia",
                "country_code": "au",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "

In [5]:
# Create an empty list to store police data
police_data = []

# Check if "features" key exists in the JSON response
if "features" in places_data:
    # Loop through all the police
    for feature in places_data["features"]:
        # Retrieve the desired information for each police
        properties = feature.get("properties", {})
        name = properties.get("name", None)
        address = properties.get("address_line2", None)
        municipality = properties.get("municipality", None)
        suburb = properties.get("suburb", None)
        postcode = properties.get("postcode", None)
        latitude = properties.get("lat", None)
        longitude = properties.get("lon", None)

        # Append the police data to the list
        police_data.append({
            "NAME": name,
            "Address": address,
            "Municipality": municipality,
            "Suburb": suburb,
            "POST_CODE": postcode,
            "Latitude":latitude,
            "Longitude":longitude
        })

    # Create a DataFrame from the police data
    police_df = pd.DataFrame(police_data)

else:
    print("No police data found.")

# Replace blank values in the "NAME" column with corresponding "Address" values
police_df["NAME"].fillna(police_df["Address"], inplace=True)

#drop rows with POST_CODE missing
police_df = police_df.dropna(subset=["POST_CODE"])


# Sort the DataFrame by the "POST_CODE" column
police_df = police_df.sort_values(by='POST_CODE')

# Reset the index and make POST_CODE the new index
police_df = police_df.set_index('POST_CODE')
police_df

,NAME,Address,Municipality,Suburb,Latitude,Longitude
POST_CODE,,,,,,
3000,Melbourne Custody Centre,"William Street, Melbourne VIC 3000, Australia",City of Melbourne,Melbourne,-37.813400,144.957000
3000,Flinders Street Station Police Booth,"207-361 Flinders Street, Melbourne VIC 3000, A...",City of Melbourne,Melbourne,-37.817967,144.967320
3000,Spencer Street Police Complex,"263-313 Spencer Street, Docklands VIC 3000, Au...",City of Melbourne,Docklands,-37.813757,144.950959
3000,Melbourne West Police Station,"313 Spencer Street, Docklands VIC 3000, Australia",City of Melbourne,Docklands,-37.813527,144.951203
3000,Melbourne Prosecutions,"456 Lonsdale Street, Melbourne VIC 3000, Austr...",City of Melbourne,Melbourne,-37.813200,144.958000
...,...,...,...,...,...,...
3936,Dromana Police Station,"Point Nepean Road, Dromana VIC 3936, Australia",Shire of Mornington Peninsula,Dromana,-38.334746,144.961068
3939,Rosebud Police Station,"95 Boneo Road, Rosebud VIC 3939, Australia",Shire of Mornington Peninsula,Rosebud,-38.369808,144.889915
3941,Rye,"Rye VIC 3941, Australia",Shire of Mornington Peninsula,Rye,-38.370600,144.819000


In [6]:
#Import mapping file - use to filter to sample postcodes for further analysis
# Specify the file path and sheet name
file_path = "Resources/Melbourne Postcodes.xlsx"
sheet_name = "Mapping"

# Read the specified sheet into a DataFrame
melb_postcodes_df = pd.read_excel(file_path, sheet_name=sheet_name)
melb_postcodes_df= melb_postcodes_df.dropna(subset=['ABS_SA2_KEY'])
melb_postcodes_df= melb_postcodes_df.dropna(subset=['HOUSE_LOCALITY'])
melb_postcodes_df= melb_postcodes_df.dropna(subset=['SCHOOL_POST_CODE'])

#convert POST_CODE to string
melb_postcodes_df['POST_CODE'] = melb_postcodes_df['POST_CODE'].astype(str)
# Display the DataFrame
melb_postcodes_df

,MUNICIPALITY,CITY_SHIRE,SUBURB_GROUP,RURAL_TOWNSHIP,POST_CODE,ABS_SA2_KEY,HOUSE_LOCALITY,SCHOOL_POST_CODE,SUBURB_NAME,COMMENTS,SUBURB_POSTCODE_COMMENTS
0,Inner City municipalities and their suburbs,City of Melbourne,Inner,NaN,3053,206041117: Carlton,CARLTON,3053.0,Carlton,NaN,Carlton 3053
2,Inner City municipalities and their suburbs,City of Melbourne,Inner,NaN,3054,206071140: Carlton North - Princes Hill,CARLTON NORTH,3054.0,Carlton North,Shared with City of Yarra,Carlton North 3054 (Shared with City of Yarra)
6,Inner City municipalities and their suburbs,City of Melbourne,Inner,NaN,3031,206031115: Flemington,FLEMINGTON,3031.0,Flemington,Shared with City of Moonee Valley,Flemington 3031 (Shared with City of Moonee Va...
7,Inner City municipalities and their suburbs,City of Melbourne,Inner,NaN,3031,206031115: Flemington,KENSINGTON,3031.0,Kensington,NaN,Kensington 3031
10,Inner City municipalities and their suburbs,City of Melbourne,Inner,NaN,3051,206041506: North Melbourne,NORTH MELBOURNE,3051.0,North Melbourne,Shared with City of Moonee Valley,North Melbourne 3051 (Shared with City of Moon...
...,...,...,...,...,...,...,...,...,...,...,...
997,Western municipalities and their suburbs,City of Wyndham,Outer,NaN,3030,213011570: Derrimut,WERRIBEE,3030.0,Werribee,NaN,Werribee 3030
1000,Western municipalities and their suburbs,City of Wyndham,Outer,NaN,3030,213011570: Derrimut,WERRIBEE SOUTH,3030.0,Werribee South,NaN,Werribee South 3030
1003,Western municipalities and their suburbs,City of Wyndham,Outer,NaN,3024,213051579: Manor Lakes - Quandong,WYNDHAM VALE,3024.0,Wyndham Vale,NaN,Wyndham Vale 3024
1005,Western municipalities and their suburbs,City of Wyndham,Outer,Rural localities,3338,213041571: Brookfield,EYNESBURY,3338.0,Eynesbury,Shared with the Shire of Melton,Eynesbury 3338 (Shared with the Shire of Melton)


In [7]:
# Concatenate SUBURB_NAME values for each POST_CODE
postcode_suburbs_df = melb_postcodes_df.groupby('POST_CODE')['SUBURB_NAME'].agg(lambda x: ', '.join(x)).reset_index()

# Display the resulting DataFrame
postcode_suburbs_df

,POST_CODE,SUBURB_NAME
0,3003,West Melbourne
1,3011,"Footscray, Seddon"
2,3012,"Brooklyn, Brooklyn, Kingsville, Maidstone, Wes..."
3,3013,"Yarraville, Aintree, Bonnie Brook"
4,3015,"Newport, Spotswood, South Kingsville"
...,...,...
188,3975,"Lynbrook, Lyndhurst"
189,3976,Hampton Park
190,3977,"Botanic Ridge, Cranbourne, Cranbourne East, Cr..."
191,3978,"Clyde, Clyde North"


In [8]:
police_clean_1_df = pd.merge(melb_postcodes_df, police_df, how='right', on='POST_CODE')

police_clean_1_df= police_clean_1_df.dropna(subset=['HOUSE_LOCALITY'])

columns_to_drop = ["RURAL_TOWNSHIP","ABS_SA2_KEY","HOUSE_LOCALITY","SCHOOL_POST_CODE",
                   "COMMENTS","SUBURB_NAME","SUBURB_POSTCODE_COMMENTS","Address","Municipality"]

police_clean_1_df = police_clean_1_df.drop(columns_to_drop, axis=1)
# Remove duplicates based on the "POST_CODE" column
police_clean_1_df = police_clean_1_df.drop_duplicates(subset='POST_CODE')

police_clean_1_df

,MUNICIPALITY,CITY_SHIRE,SUBURB_GROUP,POST_CODE,NAME,Suburb,Latitude,Longitude
7,Western municipalities and their suburbs,City of Maribyrnong,Mid,3011,Footscray Police Station,Footscray,-37.803927,144.901194
9,Western municipalities and their suburbs,City of Hobsons Bay,Mid,3016,Williamstown Police Station,Williamstown,-37.863315,144.906381
11,Western municipalities and their suburbs,City of Hobsons Bay,Mid,3018,Altona Police Station,Altona,-37.863222,144.810077
13,Western municipalities and their suburbs,City of Brimbank,Mid,3020,Sunshine Police Station,Sunshine,-37.777910,144.831231
17,Western municipalities and their suburbs,City of Brimbank,Mid,3023,Caroline Springs Police Station,Caroline Springs,-37.730629,144.741205
...,...,...,...,...,...,...,...,...
229,Southeastern municipalities and their suburbs,Shire of Mornington Peninsula,Outer,3915,Hastings Police Station,None,-38.307951,145.184646
231,Southeastern municipalities and their suburbs,Shire of Mornington Peninsula,Outer,3931,Mornington Police Station,Mornington,-38.216367,145.036895
232,Southeastern municipalities and their suburbs,Shire of Mornington Peninsula,Outer,3936,Dromana Police Station,Dromana,-38.334746,144.961068
234,Southeastern municipalities and their suburbs,Shire of Mornington Peninsula,Outer,3939,Rosebud Police Station,Rosebud,-38.369808,144.889915


In [9]:
unique_count = police_clean_1_df['POST_CODE'].nunique()

# Display the count of unique strings
print(unique_count)

84


In [10]:
police_clean_final_df = pd.merge(police_clean_1_df, postcode_suburbs_df, how='inner', on='POST_CODE')
new_column_names = {
    "SUBURB_NAME": "SUBURB_NAMES",
    }
police_clean_final_df = police_clean_final_df.rename(columns=new_column_names)

# Reset the index and make POST_CODE the new index
police_clean_final_df = police_clean_final_df.set_index('POST_CODE')

# Export cleaned population dataset to a CSV file
police_clean_final_df.to_csv("Cleaned_Data/police_clean_final.csv", index_label="POST_CODE")

police_clean_final_df

,MUNICIPALITY,CITY_SHIRE,SUBURB_GROUP,NAME,Suburb,Latitude,Longitude,SUBURB_NAMES
POST_CODE,,,,,,,,
3011,Western municipalities and their suburbs,City of Maribyrnong,Mid,Footscray Police Station,Footscray,-37.803927,144.901194,"Footscray, Seddon"
3016,Western municipalities and their suburbs,City of Hobsons Bay,Mid,Williamstown Police Station,Williamstown,-37.863315,144.906381,"Williamstown, Williamstown North"
3018,Western municipalities and their suburbs,City of Hobsons Bay,Mid,Altona Police Station,Altona,-37.863222,144.810077,"Altona, Seaholme"
3020,Western municipalities and their suburbs,City of Brimbank,Mid,Sunshine Police Station,Sunshine,-37.777910,144.831231,"Albion, Sunshine, Sunshine North, Sunshine West"
3023,Western municipalities and their suburbs,City of Brimbank,Mid,Caroline Springs Police Station,Caroline Springs,-37.730629,144.741205,"Cairnlea, Deer Park, Burnside, Burnside Height..."
...,...,...,...,...,...,...,...,...
3915,Southeastern municipalities and their suburbs,Shire of Mornington Peninsula,Outer,Hastings Police Station,None,-38.307951,145.184646,"Hastings, Hastings West"
3931,Southeastern municipalities and their suburbs,Shire of Mornington Peninsula,Outer,Mornington Police Station,Mornington,-38.216367,145.036895,Mornington
3936,Southeastern municipalities and their suburbs,Shire of Mornington Peninsula,Outer,Dromana Police Station,Dromana,-38.334746,144.961068,"Dromana, Safety Beach"


In [11]:
# Group the hospitals_clean_final_df by the specified columns and count the occurrences of NAME
police_count_by_postcode_df = police_clean_final_df.groupby(['POST_CODE', 'MUNICIPALITY', 'CITY_SHIRE', 'SUBURB_GROUP'])['NAME'].count().reset_index()

# Rename the count column to "Hospital_Count"
police_count_by_postcode_df = police_count_by_postcode_df.rename(columns={'NAME': 'Police_Count'})

# Export the hospital count by postcode to a CSV file
police_count_by_postcode_df.to_csv("Cleaned_Data/police_count_by_postcode.csv", index=False)

# Display the hospital count DataFrame
police_count_by_postcode_df

,POST_CODE,MUNICIPALITY,CITY_SHIRE,SUBURB_GROUP,Police_Count
0,3011,Western municipalities and their suburbs,City of Maribyrnong,Mid,1
1,3016,Western municipalities and their suburbs,City of Hobsons Bay,Mid,1
2,3018,Western municipalities and their suburbs,City of Hobsons Bay,Mid,1
3,3020,Western municipalities and their suburbs,City of Brimbank,Mid,1
4,3023,Western municipalities and their suburbs,City of Brimbank,Mid,1
...,...,...,...,...,...
79,3915,Southeastern municipalities and their suburbs,Shire of Mornington Peninsula,Outer,1
80,3931,Southeastern municipalities and their suburbs,Shire of Mornington Peninsula,Outer,1
81,3936,Southeastern municipalities and their suburbs,Shire of Mornington Peninsula,Outer,1
82,3939,Southeastern municipalities and their suburbs,Shire of Mornington Peninsula,Outer,1
